In [1]:
import xarray as xr
from xgcm import Grid
import numpy as np

In [2]:
rootdir = '/local/data/bSOSE/'
niter = 'iter133NEW'
freq = '5day'
# Load all carbon tendencies and fluxes
filenames = 'bsose_i133_2013to2018_5day_*C.nc'
ds = xr.open_mfdataset(rootdir+niter+'/'+freq+'/'+filenames)
# Load carbon snapshots
filenames = 'bsose_i133_2013to2018_5daySnapShots_DIC.nc'
ds_snaps = xr.open_dataset(rootdir+niter+'/'+freq+'/'+filenames)
# ds_snaps = ds_snaps.rename({'time':'time_snaps','TRAC01':'TRAC01_snaps'}).drop('iter')
# # Specify shift of time axis
# ### NOTE: check if the shift _needs_ to be 15 days (not 0.5)
# ds_snaps['time_snaps'].attrs['c_grid_axis_shift']=-0.5
# # Merge to full dataset
# ds = xr.merge([ds,ds_snaps])

# # Define vertical metrics as negative, to account for descending coordinate
# ds['drW'] = -1 * ds.hFacW * ds.drF #vertical cell size at u point
# ds['drS'] = -1 * ds.hFacS * ds.drF #vertical cell size at v point
# ds['drC'] = -1 * ds.hFacC * ds.drF #vertical cell size at tracer point
# ds['vC'] = ds['drC']*ds['rA']

# metrics = {
#     ('X',): ['dxC', 'dxG'], # X distances
#     ('Y',): ['dyC', 'dyG'], # Y distances
#     ('Z',): ['drW', 'drS', 'drC'], # Z distances
#     ('X', 'Y'): ['rA', 'rAs', 'rAw'] # Areas
# }

# xgrid = Grid(ds,periodic=['X'],metrics=metrics)
# xgrid

In [ ]:
ds_snaps['TRAC01'].diff('Z')

In [6]:
ds['TRAC01_snaps'].diff('time_snaps')

KeyboardInterrupt: 

In [ ]:
xgrid.diff(ds['TRAC01_snaps'],'T')

In [6]:
budget = xr.Dataset()

# budget['TEND'] = xgrid.diff(ds['TRAC01_snaps'],'T')

budget['ADV'] = -(xgrid.diff(ds['ADVxTr01'],'X')+
                  xgrid.diff(ds['ADVyTr01'],'Y',boundary='extend')+
                  xgrid.diff(ds['ADVrTr01'],'Z',boundary='extend'))/ds['vC']

budget['DIFF'] = -(xgrid.diff(ds['DFxETr01'],'X')+
                  xgrid.diff(ds['DFyETr01'],'Y',boundary='extend')+
                  xgrid.diff(ds['DFrITr01'],'Z',boundary='extend'))/ds['vC']

budget['BIO'] = ds['BLGBIOC']

budget['FORC'] = ds['ForcTr01']

budget['CORR'] = ds['WTRAC01']


In [5]:
ds['time']

<xarray.Dataset>
Dimensions:   (XC: 2160, XG: 2160, YC: 588, YG: 588, Z: 52, Zl: 52, time: 438)
Coordinates: (12/20)
    iter      (time) int64 dask.array<chunksize=(438,), meta=np.ndarray>
  * time      (time) datetime64[ns] 2013-01-05 2013-01-10 ... 2018-12-30
  * XC        (XC) float32 0.08333 0.25 0.4167 0.5833 ... 359.6 359.8 359.9
  * YC        (YC) float32 -77.98 -77.95 -77.91 -77.88 ... -30.02 -29.87 -29.72
  * Zl        (Zl) float32 0.0 -4.2 -9.2 -15.1 ... -4.8e+03 -5.2e+03 -5.6e+03
    Depth     (YC, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    ...        ...
  * YG        (YG) float32 -78.0 -77.97 -77.93 -77.9 ... -30.09 -29.95 -29.8
    rAs       (YG, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    dxG       (YG, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    dyC       (YG, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    hFacS     (Z, YG, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
    hFacC     (Z, YC, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
Data variables: (12/14)
    ADVrTr01  (time, Zl, YC, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    ADVxTr01  (time, Z, YC, XG) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    ADVyTr01  (time, Z, YG, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    BLGBIOC   (time, Z, YC, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    DFrITr01  (time, Zl, YC, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    DFxETr01  (time, Z, YC, XG) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    ...        ...
    TRAC01    (time, Z, YC, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    WTRAC01   (time, Z, YC, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    drW       (Z, YC, XG) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
    drS       (Z, YG, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
    drC       (Z, YC, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
    vC        (Z, YC, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>

In [32]:
ds

<xarray.Dataset>
Dimensions:   (XC: 2160, XG: 2160, YC: 588, YG: 588, Z: 52, Zl: 52, time: 438)
Coordinates: (12/20)
    iter      (time) int64 dask.array<chunksize=(438,), meta=np.ndarray>
  * time      (time) datetime64[ns] 2013-01-05 2013-01-10 ... 2018-12-30
  * XC        (XC) float32 0.08333 0.25 0.4167 0.5833 ... 359.6 359.8 359.9
  * YC        (YC) float32 -77.98 -77.95 -77.91 -77.88 ... -30.02 -29.87 -29.72
  * Zl        (Zl) float32 0.0 -4.2 -9.2 -15.1 ... -4.8e+03 -5.2e+03 -5.6e+03
    Depth     (YC, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    ...        ...
  * YG        (YG) float32 -78.0 -77.97 -77.93 -77.9 ... -30.09 -29.95 -29.8
    rAs       (YG, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    dxG       (YG, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    dyC       (YG, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    hFacS     (Z, YG, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
    hFacC     (Z, YC, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
Data variables: (12/13)
    ADVrTr01  (time, Zl, YC, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    ADVxTr01  (time, Z, YC, XG) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    ADVyTr01  (time, Z, YG, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    DFrITr01  (time, Zl, YC, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    DFxETr01  (time, Z, YC, XG) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    DFyETr01  (time, Z, YG, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    ...        ...
    TRAC01    (time, Z, YC, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    WTRAC01   (time, Z, YC, XC) float32 dask.array<chunksize=(438, 52, 588, 2160), meta=np.ndarray>
    drW       (Z, YC, XG) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
    drS       (Z, YG, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
    drC       (Z, YC, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
    vC        (Z, YC, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>

In [10]:
ds_snaps.time

<xarray.DataArray 'time' (time: 439)>
array(['2012-12-31T00:00:00.000000000', '2013-01-05T00:00:00.000000000',
       '2013-01-10T00:00:00.000000000', ..., '2018-12-20T00:00:00.000000000',
       '2018-12-25T00:00:00.000000000', '2018-12-30T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
    iter     (time) int64 2160 2520 2880 3240 ... 158760 159120 159480 159840
  * time     (time) datetime64[ns] 2012-12-31 2013-01-05 ... 2018-12-30
Attributes:
    long_name:      Time
    standard_name:  time
    axis:           T